In [6]:
import pyodbc
import pandas as pd


### Database connection configuration

In [7]:
DB_CONFIG = {
    "server": "localhost",  # Your SQL Server address
    "database": "Walmart",  # Your database name
    "username": "sa",       # Your SQL Server username
    "password": "Type_your_own_password_here="  # Your SQL Server password
}

In [10]:
def connect_to_db(db_config):
    """Establish a connection to the SQL Server."""
    print("Connecting to the database...")
    conn = pyodbc.connect(
        f"DRIVER={{ODBC Driver 17 for SQL Server}};"
        f"SERVER={db_config['server']};"
        f"DATABASE={db_config['database']};"
        f"UID={db_config['username']};"
        f"PWD={db_config['password']}"
    )
    print("Connection successful!")
    return conn

def fetch_data(cursor, table_name):
    """Fetch data from the table and handle potential issues."""
    print(f"Fetching data from the {table_name} table...")
    query = f"SELECT * FROM {table_name}"
    cursor.execute(query)
    rows = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    
    # Convert rows to DataFrame
    rows_as_lists = [list(row) for row in rows]  # Convert each row to a list
    df = pd.DataFrame(rows_as_lists, columns=columns)
    
    print(f"Fetched {len(df)} rows and {len(columns)} columns from the {table_name} table.")
    return df

def clean_data(df):
    """Perform cleaning operations on the data."""
    print("Cleaning data...")

    # Example: Remove rows with missing values (NaN)
    initial_row_count = len(df)
    df = df.dropna()  # Drop rows with any NaN values
    print(f"Removed {initial_row_count - len(df)} rows with missing values.")

    # Example: Ensure 'float' columns contain only valid numbers
    float_columns = ['product_name_length', 'product_description_length', 'product_photos_qty', 
                     'product_weight_g', 'product_length_cm', 'product_height_cm', 'product_width_cm']
    
    for col in float_columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')  # Convert to numeric, invalid values become NaN

    # Example: Replace NaN values in 'float' columns with 0 or another default value
    df[float_columns] = df[float_columns].fillna(0)
    print(f"Cleaned {len(df)} rows. NaN values in numeric columns replaced with 0.")
    
    print("Data cleaning complete.")
    return df

def update_data_bulk(cursor, df, table_name):
    """Update the cleaned data back into the SQL table using batch updates."""
    print(f"Updating data in the {table_name} table (bulk update)...")
    
    update_query = """
    UPDATE {table_name} 
    SET 
        product_category = ?, 
        product_name_length = ?, 
        product_description_length = ?, 
        product_photos_qty = ?, 
        product_weight_g = ?, 
        product_length_cm = ?, 
        product_height_cm = ?, 
        product_width_cm = ? 
    WHERE product_id = ?
    """.format(table_name=table_name)

    # Prepare the data as a list of tuples for bulk execution
    update_values = [tuple(row) for _, row in df.iterrows()]
    
    cursor.executemany(update_query, update_values)
    print(f"Successfully updated {len(df)} records in the {table_name} table.")

def main():
    print("Starting process...")

    # Connect to the database
    conn = connect_to_db(DB_CONFIG)
    cursor = conn.cursor()

    # Fetch data from the `products` table
    table_name = "products"
    df = fetch_data(cursor, table_name)

    # Clean the data
    cleaned_df = clean_data(df)

    # Update cleaned data back to SQL Server (bulk update)
    update_data_bulk(cursor, cleaned_df, table_name)
    
    # Commit the changes and close the connection
    conn.commit()
    conn.close()

    print("Data cleaned and updated successfully.")
    print("Process complete.")




### Run the process

In [9]:

main()